In [1]:
import numpy as np
from sklearn.datasets import make_blobs

In [3]:
X, y = make_blobs(n_samples=1000, centers=2, random_state=200)

In [5]:
import torch
import torch.nn as nn

In [6]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
y = y*2 - 1

In [7]:
class LinearSVC():
    def __init__(self, input_dim, lambd=1e-4, batch_size=100, lr=0.1, iters=10000):
        self.input_dim = input_dim
        self.lambd = lambd
        self.batch_size = batch_size
        self.lr = lr
        self.iters = iters
        self.w = nn.Parameter(torch.randn(input_dim))
        self.b = nn.Parameter(torch.randn(1))
    
    def _loss(self, x, y):
        out = 1 - y * (torch.matmul(x, self.w) + self.b)
        out = torch.clamp(out, min=0)
        loss = 0.5 * self.lambd * torch.norm(self.w)**2 + out.mean()
        return loss
    
    def _zero_grad(self):
        if self.w.grad is not None:
            self.w.grad.zero_()
            self.b.grad.zero_()
    
    def _update_weights(self):
        self.w.data -= self.lr * self.w.grad.data
        self.b.data -= self.lr * self.b.grad.data
    
    def fit(self, data, labels):
        N = len(data)
        for i in range(self.iters):
            idx = np.random.choice(N, size=self.batch_size, replace=False)
            batch = data[idx]
            label = labels[idx]
            self._zero_grad()
            loss = self._loss(batch, label)
            loss.backward()
            self._update_weights()
    
    def predict(self, data):
        with torch.no_grad():
            score = torch.matmul(data, self.w.data) + self.b.data
            pred = (score > 0).to(data.dtype)
            pred = pred * 2 - 1
        
        return pred, score

In [8]:
clf = LinearSVC(2)

In [9]:
clf.fit(X, y)

In [10]:
pred, score = clf.predict(X)